In [1]:
import json
import numpy as np
from numpy.matlib import repmat
from scipy.spatial import distance_matrix as ssdm
import matplotlib.pyplot as plt
import pdb
import glob
from PIL import Image
import cv2

import carla 

from identify_instances import COLORMAP
%matplotlib

Using matplotlib backend: Qt5Agg


# Data Folder Selection

In [2]:
town_str = 'Town02' # 01, 02, 04
datadir = '../../%s_demo/' % town_str

# Image and Snapshot Preparation Helper Functions

In [3]:
def process_rgb_image(imgloc):
    im_rgb = Image.open(imgloc).convert('RGB')
    return np.array(im_rgb)

def process_seg_image(imgloc):
    im_seg = Image.open(imgloc)
    return np.array(im_seg)[:,:,0]

def process_depth_image(imgloc):
    im_depth = Image.open(imgloc)
    im_depth = np.array(im_depth)[:,:,:3].astype(np.float)
    
    # formula: 
    # norm_depth = (r + g * 256 + b * 256**2) / (256**3 - 1)
    # norm_depth = 1 => 1 km depth in optical axis

    # https://github.com/carla-simulator/carla/issues/2817
    norm_depth = np.dot(im_depth, [1.0, 256.0, 65536.0])
    norm_depth /= 16777215.0
    
    im_depth = norm_depth * 1000. # meters, max resolution is 1 km
    return im_depth

def process_snapshot_json(jsonfile):
    data_dict = json.load(open(jsonfile, 'r'))
    return data_dict

# Point Cloud Processing Helper Function

In [4]:
def project_camera_point_cloud(seg_img, depth_img, cx, cy, f):
    # Vectorized implemention courtesy of:
    # https://github.com/carla-simulator/carla/blob/cffd944c466026a1fca1ea1f711b86c96117acee/PythonClient/carla/image_converter.py#L111
    K = np.array([[f, 0., cx], \
                  [0., f, cy], \
                  [0., 0., 1]], dtype=np.float)
    
    height, width = seg_img.shape
    
    us = repmat(np.r_[0:width:1], height, 1)
    vs = repmat(np.c_[0:height:1], 1, width)
    
    p2d_list = np.array([us.flatten(), vs.flatten(), np.ones(height*width)], dtype=np.float) # 3 by HW, homogenous 2d point representation    
    p3d_list = depth_img.flatten() * np.dot(np.linalg.inv(K), p2d_list) # 3 by HW, XYZ camera 3D coordinates
    
    color_list = np.array([COLORMAP[col] for col in seg_img.flatten()], dtype=np.float) # i.e. 0.0 -> 255.0
    seg_lbl = seg_img.flatten().reshape(width*height, 1)
    
    point_cloud_list = np.concatenate((p3d_list.T, seg_lbl, color_list), axis=1) # [X, Y, Z, seg_label, col_r, col_g, col_b]
    return point_cloud_list

def threshold_point_cloud_depth(point_cloud_list, max_depth=100.):
    # Remove points with depth (Z) greater than max_depth.
    keep_inds = point_cloud_list[:,2] < max_depth
    return point_cloud_list[keep_inds,:]

def threshold_point_cloud_seg_class(point_cloud_list, seg_classes=[4, 10]):
    # Remove points that have a segmentation label not in seg_classes.
    keep_inds = np.zeros(point_cloud_list.shape[0], dtype=np.bool)
    
    for seg_lbl in seg_classes:
        match_inds = point_cloud_list[:,3] == seg_lbl
        keep_inds += match_inds # OR
    
    return point_cloud_list[keep_inds,:]
    
"""
def project_camera_points_by_class(seg_img, depth_img, fov, seg_classes = [4]):
    # Converts instances with label in seg_classes from pixel to 3D.  Final transform
    # used to convert from camera coordinates to another 3D world coordinate system.
    
    height, width = seg_img.shape
    cx = width / 2.
    cy = height / 2.
    f  = width / ( 2. * np.tan(np.radians(fov/2.)) )    
    cam_3d_points = []
    
    for seg_lbl in seg_classes:
        seg_inds = np.argwhere(seg_img == seg_lbl)
        seg_color = COLORMAP[seg_lbl]
        
        for (v,u) in seg_inds:            
            Z_cam = depth_img[v,u]
            X_cam = (u - cx + 0.5) / f * Z_cam
            Y_cam = (v - cy + 0.5) / f * Z_cam
            
            
            
            if final_transform:
                X_unreal_local, Y_unreal_local, Z_unreal_local = \
                    convert_pinhole_to_unreal_3d_coords([X_cam, Y_cam, Z_cam])
                
                loc_unreal_global = local_to_world_frame(carla.Location(x=X_unreal_local,
                                                                        y=Y_unreal_local,
                                                                        z=Z_unreal_local), \
                                                         final_transform)
                
                cam_3d_points.append([loc_unreal_global[0],
                                      loc_unreal_global[1],
                                      loc_unreal_global[2],
                                      seg_color[0],
                                      seg_color[1],
                                      seg_color[2]])
            else:
                cam_3d_points.append([X_cam, Y_cam, Z_cam, \
                                      seg_color[0], seg_color[1], seg_color[2]])
    return np.array(cam_3d_points)

def project_camera_point_cloud(seg_img, depth_img, fov):
    # This function converts a scene represented by segmentation and depth images
    # to a point cloud array.
    
    height, width = seg_img.shape
    cx = width / 2.
    cy = height / 2.
    f  = width / ( 2. * np.tan(np.radians(fov/2.)) )
    
    cam_point_cloud = []
    
    # TODO: vectorize and prune elements with 0 label at the end.
    # reference: https://github.com/carla-simulator/carla/blob/cffd944c466026a1fca1ea1f711b86c96117acee/PythonClient/carla/image_converter.py#L148
    for v in range(int(cy), height, 2):
        for u in range(0, width, 1):            
            lbl = seg_img[v,u]
            if lbl == 0:
                continue
                
            Z_cam = depth_img[v,u]
            X_cam = (u - cx + 0.5) / f * Z_cam
            Y_cam = (v - cy + 0.5) / f * Z_cam
            
            color = COLORMAP[lbl]
            
            cam_point_cloud.append([X_cam, Y_cam, Z_cam, color[0], color[1], color[2]])
    
    return np.array(cam_point_cloud)
"""

def downsample_and_project_point_cloud(point_cloud, xmin_m, xmax_m, zmin_m, zmax_m, grid_res):
    # Rough top-view projection of the point cloud.  X,Z are in the camera coordinate system.
    
    # Assuming a grid with grid resolution and limits (*min, *max), this returns the center of each cell
    # in the grid.  This can then be used to populate the cell with pixel RGB values based on a
    # nearest neighbor downsampling.
    
    # Grid Coordinates in X and Z.
    xrow = np.r_[xmin_m + grid_res / 2. : xmax_m : grid_res] # N
    zcol = np.c_[zmin_m + grid_res / 2. : zmax_m : grid_res] # M
    
    # Number of Grid Coordinates by Axis (N, M).
    N_x = np.size(xrow)
    M_z = np.size(zcol)

    # Top View projection Image.
    projection = np.zeros((M_z, N_x, 3), dtype=np.uint8) # M by N
    
    # Grid XZ points.
    xs = repmat(xrow, M_z, 1) # M by N
    zs = repmat(zcol, 1, N_x) # M by N
    
    # XZ Grid and Point Cloud List.
    xz_grid_list = np.column_stack( (xs.flatten(), zs.flatten()) ) # MN by 2
    xz_point_cloud = point_cloud[:, [0,2]]
    
    # Euclidean Distance Matrix.
    # # dm[i,j] = dist(xz_grid_list[i], xz_point_cloud[j])
    min_inds = np.ones(xz_grid_list.shape[0]) * -1
    
    print('SSDM Started')
    
    for st_ind in np.arange(0, xz_grid_list.shape[0], 1000):
        end_ind = min(xz_grid_list.shape[0], st_ind + 1000)
        dm = ssdm(xz_grid_list[st_ind:end_ind,:], xz_point_cloud) 
        
        min_dist_subset = np.amin(dm, axis=1)   # dist from xz grid location to nearest point cloud point
        min_inds_subset = np.argmin(dm, axis=1) # index of nearest point cloud point
        min_inds_subset[min_dist_subset > grid_res] = -1 # no point nearby
        min_inds[st_ind:end_ind] = min_inds_subset        
        
    print('SSDM Done')
    
    # Populate projection image with colors.
    min_inds = min_inds.reshape( (M_z, N_x) )
    for ind_z in range(min_inds.shape[0]):
        for ind_x in range(min_inds.shape[1]):
            if min_inds[ind_z, ind_x] < 0:
                continue
            else:
                pc_ind = int(min_inds[ind_z, ind_x])
                projection[ind_z, ind_x] = point_cloud[ pc_ind, 4:].astype(np.uint8)
    
    print('For Loop Done')
    
    return projection

# Plot 3D Elements Helper Functions

In [5]:
def plot_orientation_arrow(origin, yaw, length, color='c', width=0.1):
    x, y, = origin
    dx = length * np.cos( np.radians(yaw) )
    dy = length * np.sin( np.radians(yaw) )
    plt.arrow(x, y, dx, dy, color=color, width=width)

def plot_text(origin, text, color='c'):
    x, y = origin
    plt.text(x, y + 0.2, text, color=color, fontsize='medium')
    
    
def plot_bbox_world_vertices(world_vertices):
    bottom_left_back  = world_vertices[0]
    bottom_right_back = world_vertices[2]
    bottom_right_fwd  = world_vertices[6]
    bottom_left_fwd   = world_vertices[4]
    
    plt.plot([bottom_left_back['x'], bottom_right_back['x']], \
             [bottom_left_back['y'], bottom_right_back['y']], 'k')
    
    plt.plot([bottom_right_back['x'], bottom_right_fwd['x']], \
             [bottom_right_back['y'], bottom_right_fwd['y']], 'r')
    
    plt.plot([bottom_right_fwd['x'], bottom_left_fwd['x']], \
             [bottom_right_fwd['y'], bottom_left_fwd['y']],'g')
    
    plt.plot([bottom_left_fwd['x'], bottom_left_back['x']], \
             [bottom_left_fwd['y'], bottom_left_back['y']], 'b')
    
    [plt.plot(p['x'], p['y'], 'k.') for p in world_vertices]

def get_rotation_matrix(transform):
    # Source: https://github.com/carla-simulator/carla/blob/3926fec5fb448b6c52b5a2d5e73c790e8736cc11/LibCarla/source/carla/geom/Transform.h#L81
    # Also helpful: https://github.com/carla-simulator/carla/blob/dev/LibCarla/source/carla/geom/Math.cpp
    # This is implemented in the latest Carla versions but not 0.9.9 :(
    # Thus replicated here for convenience.
    # NOTE: Unreal coordinate system (LHS).
    
    cp = np.cos(np.radians(transform.rotation.pitch))
    sp = np.sin(np.radians(transform.rotation.pitch))
    
    cr = np.cos(np.radians(transform.rotation.roll))
    sr = np.sin(np.radians(transform.rotation.roll))
    
    cy = np.cos(np.radians(transform.rotation.yaw))
    sy = np.sin(np.radians(transform.rotation.yaw))
    
    rotation_matrix = np.array([\
        [cp * cy,  cy * sp * sr - sy * cr, -cy * sp * cr - sy * sr], \
        [cp * sy,  sy * sp * sr + cy * cr, -sy * sp * cr + cy * sr], \
        [sp     , -cp * sr              ,   cp * cr]])
    return rotation_matrix
    
def world_to_local_frame(world_loc, transform):
    # NOTE: Unreal coordinate system (LHS).
    if type(world_loc) == carla.Location:
        diff_loc = world_loc - transform.location
        diff_arr = np.array([diff_loc.x, diff_loc.y, diff_loc.z]).reshape(3,1)
    else:
        trans_arr = np.array([transform.location.x, transform.location.y, transform.location.z]).reshape(3,1)
        diff_arr = world_loc - trans_arr        
    
    inv_rotation_matrix = get_rotation_matrix(transform).T
    local_loc_arr = inv_rotation_matrix @ diff_arr
   
    return local_loc_arr

def local_to_world_frame(local_loc, transform):
    # NOTE: Unreal coordinate system (LHS).
    rotation_matrix = get_rotation_matrix(transform)
    trans_arr = np.array([transform.location.x, transform.location.y, transform.location.z]).reshape(3,1)
    
    if type(local_loc) == carla.Location:
        local_arr = np.array([local_loc.x, local_loc.y, local_loc.z]).reshape(3,1)
    else:
        pass
    
    rot_vec = rotation_matrix @ local_arr
    world_loc_arr = rot_vec + trans_arr
    
    return world_loc_arr
    
def convert_pinhole_to_unreal_3d_coords(coord_pinhole_3d):
    """ Assumptions:
        - coord_pinhole_3d are the camera coordinates used by the pinhole model (right-handed):
          X_cam=right, Y_cam=down, Z_cam=fwd
        - Unreal uses a left-handed coordinate system, according to the following link:
        http://www.aclockworkberry.com/world-coordinate-systems-in-3ds-max-unity-and-unreal-engine/
          X_unreal = fwd, Y_unreal=right, Z_unreal=up
    """
    
    X_cam, Y_cam, Z_cam = coord_pinhole_3d
    return Z_cam, X_cam, -Y_cam

def project_world_boxes_in_camera_frame(world_bbox, camera_transform, width, height, fov):    
    cx = width / 2.
    cy = height / 2.
    f  = width / ( 2. * np.tan(np.radians(fov/2.)) )
    
    us = []; vs = []
    for ind, world_vertex in enumerate(world_bbox['world_vertices']):
        print(ind)
        print('\tX: ', world_vertex['x'], \
              'Y: ', world_vertex['y'], \
              'Z: ', world_vertex['z'])
        
        local_loc = world_to_local_frame(carla.Location(x=world_vertex['x'], \
                                                        y=world_vertex['y'], \
                                                        z=world_vertex['z']), \
                                         camera_transform)
        
        # TODO: inverse of convert_pinhole..., maybe make a function later.
        X_cam, Y_cam, Z_cam = local_loc[1], -local_loc[2], local_loc[0]
        
        print('\tX_cam: ', X_cam, \
              'Y_cam: ', Y_cam, \
              'Z_cam: ', Z_cam)
        
        u = f * X_cam / Z_cam + cx
        v = f * Y_cam / Z_cam + cy
        us.append(u)
        vs.append(v)
        
        print('\t pixel:', u, v)
    return us, vs
#     print('\t', np.min(us), np.min(vs), np.max(us), np.max(vs), '\n')
#     return np.min(us), np.min(vs), np.max(us), np.max(vs) # umin, vmin, umax, vmax

# Data Loading

In [6]:
data_dicts  = [process_snapshot_json(x) for x in glob.glob(datadir + '/snapshot/*.json')]

center_imgs = [[x for x in glob.glob(datadir + '/rgb_center/*.png')], \
               [x for x in glob.glob(datadir + '/seg_center/*.png')], \
               [x for x in glob.glob(datadir + '/depth_center/*.png')]]

right_imgs  = [[x for x in glob.glob(datadir + '/rgb_right/*.png')], \
               [x for x in glob.glob(datadir + '/seg_right/*.png')], \
               [x for x in glob.glob(datadir + '/depth_right/*.png')]]

left_imgs   = [[x for x in glob.glob(datadir + '/rgb_left/*.png')], \
               [x for x in glob.glob(datadir + '/seg_left/*.png')], \
               [x for x in glob.glob(datadir + '/depth_left/*.png')]]

far_imgs    = [[x for x in glob.glob(datadir + '/rgb_far/*.png')], \
               [x for x in glob.glob(datadir + '/seg_far/*.png')], \
               [x for x in glob.glob(datadir + '/depth_far/*.png')]]

dataset_lengths = [len(data_dicts)]
for img_dict in [center_imgs, right_imgs, left_imgs, far_imgs]:
    dataset_lengths.extend([len(x) for x in img_dict])
length_consistency = np.all([x == dataset_lengths[0] for x in dataset_lengths])
print('Length Consistency Check: %s with %d elements each.' % (length_consistency, dataset_lengths[0]))

Length Consistency Check: True with 50 elements each.


In [7]:
#[print(k) for k in data_dicts[0].keys()]

def load_local_scene(data_dict, ego_name='hero'):
    ego_dict           = {}
    other_vehicle_dict = {}
    walker_dict        = {}
    traffic_light_dict = {}
    traffic_stop_dict  = {}
    traffic_yield_dict = {}
    traffic_speed_dict = {}
    
    for (k, v) in data_dict.items():
        if k in ['id', 'frame', 'delta_seconds', 'elapsed_seconds']:
            continue
        
        actor_id = int(k.split('_')[-1])
        
        if 'traffic_light' in k:
            traffic_light_dict[actor_id] = v
        elif 'traffic.stop' in k:
            traffic_stop_dict[actor_id]  = v
        elif 'traffic.yield' in k:
            traffic_yield_dict[actor_id] = v
        elif 'traffic.speed_limit' in k:
            speed = int(k.split('speed_limit.')[-1].split('_')[0])
            traffic_speed_dict[actor_id] = v
            traffic_speed_dict[actor_id]['speed'] = speed
        elif 'vehicle' in k:
            if v['role_name'] == 'hero':
                ego_dict = v
                ego_dict['make'] = k.split('vehicle.')[-1].split('_')[0]
            else:
                other_vehicle_dict[actor_id] = v
                other_vehicle_dict[actor_id]['make'] = k.split('vehicle.')[-1].split('_')[0]
        elif 'walker' in k:
            walker_dict[actor_id] = v
    
    scene_dict = {}
    scene_dict['ego'] = ego_dict
    scene_dict['other_vehicles'] = other_vehicle_dict
    scene_dict['walkers'] = walker_dict
    scene_dict['traffic_lights'] = traffic_light_dict
    scene_dict['stop_signs'] = traffic_stop_dict
    scene_dict['yield_signs'] = traffic_yield_dict
    scene_dict['speed_signs'] = traffic_speed_dict
    
    return scene_dict

def convert_carla_location(carla_loc_dict):
    # https://github.com/carla-simulator/ros-bridge/blob/0556f05d086b6d43c6f91f073d116654b3c84969/carla_common/src/carla_common/transforms.py#L51
    x, y, z = [carla_loc_dict[k] for k in ['x', 'y', 'z']]
    return np.array([x, -y, z]).reshape(3,1)

def convert_carla_rotation(carla_rot_dict):
    # https://github.com/carla-simulator/ros-bridge/blob/0556f05d086b6d43c6f91f073d116654b3c84969/carla_common/src/carla_common/transforms.py#L109
    
    r, p, y = [carla_rot_dict[k] for k in ['roll', 'pitch', 'yaw']]
    return np.radians([r, -p, -y])

def euler_to_rotation_matrix(r, p, y):
    # This is using the ROS convention!  It rotates about x, then y, then z using static axes.
    # R = R_z(yaw_ros) * R_y(pitch_ros) * R_x(roll_ros).
    # Source: http://docs.ros.org/jade/api/tf/html/python/transformations.html (default = 'sxyz')
    c = np.cos
    s = np.sin
    
    Rx_r = np.array([[1, 0, 0], [0, c(r), -s(r)], [0, s(r), c(r)]], dtype=np.float)
    Ry_p = np.array([[c(p), 0, s(p)], [0, 1, 0], [-s(p), 0, c(p)]], dtype=np.float)
    Rz_y = np.array([[c(y), -s(y), 0], [s(y), c(y), 0], [0, 0, 1]], dtype=np.float)
    
    return Rz_y @ Ry_p @ Rx_r

def local_to_world_frame(v3d_local, R, t):
    # R: local -> world
    # t: local's origin in world coordinates
    return R @ v3d_local + t
    

def world_to_local_frame(v3d_world, R, t):
    # R: local -> world
    # t: local's origin in world coordinates
    R_inv = R.T
    t_inv = - R.T @ t
    return R_inv @ v3d_world + t_inv
    
def plot_local_scene_ego_view(scene_dict):
    # Notation: *_w => in a right-handed coordinate system corresponding to the world system
    #           *_e => in a right-handed coordinate system corresponding to ego's reference
    ego_loc_w = convert_carla_location(scene_dict['ego']['location'])
    ego_rpy_w = convert_carla_rotation(scene_dict['ego']['rotation'])
    ego_R_w = euler_to_rotation_matrix(*ego_rpy_w)
    
    plt.figure()    
    
    plt.arrow(0, 0, 10, 0, color='k')
    plt.arrow(0, 0, 0, 10, color='g')
    
    for (k,v) in scene_dict['other_vehicles'].items():
        veh_loc_w = convert_carla_location(v['location'])
        veh_loc_e = world_to_local_frame(veh_loc_w, ego_R_w, ego_loc_w)        
        
        x_e, y_e, z_e = veh_loc_e
        plt.plot(x_e, y_e, 'b.')
        plt.text(x_e, y_e - 0.5, 'veh_%d' % v['id'], color='b')
    
    for (k,v) in scene_dict['walkers'].items():
        ped_loc_w = convert_carla_location(v['location'])
        ped_loc_e = world_to_local_frame(ped_loc_w, ego_R_w, ego_loc_w)        
        
        x_e, y_e, z_e = ped_loc_e
        plt.plot(x_e, y_e, 'r.')
        plt.text(x_e, y_e - 0.5, 'ped_%d' % v['id'], color='r')
        
    plt.xlim([-10, 75])
    plt.ylim([-50, 50])
        
    
        
        
        
        
        
        

scene_dict = load_local_scene(data_dicts[25])
plot_local_scene_ego_view(scene_dict)

In [12]:
camera_arr =  center_imgs
fov = 120.
img_num    = 25
camera_role_name = 'center'
camera_transform = None

for k,v in data_dicts[img_num].items():
    if 'rgb' in k:
        if camera_role_name == v['role_name']:
            
            loc = carla.Location(x = v['location']['x'],
                                 y = v['location']['y'],
                                z = v['location']['z'])
            rot = carla.Rotation(pitch = v['rotation']['pitch'],
                                 yaw   = v['rotation']['yaw'],
                                 roll  = v['rotation']['roll'])
            
            camera_transform = carla.Transform(location=loc, rotation=rot)

print(camera_transform)

rgb_img   = process_rgb_image(camera_arr[0][img_num])
seg_img   = process_seg_image(camera_arr[1][img_num])
depth_img = process_depth_image(camera_arr[2][img_num])

plt.figure()
plt.subplot(131)
plt.imshow(rgb_img)
plt.subplot(132)
plt.imshow(seg_img, cmap='jet')
plt.subplot(133)
plt.imshow(np.log(depth_img), cmap='gray')

height, width = seg_img.shape
cx = float(width / 2)
cy = float(height / 2)
f  = width / ( 2. * np.tan(np.radians(fov/2.)) )  

plt.figure()
print('Projecting')
point_cloud = project_camera_point_cloud(seg_img, depth_img, cx, cy, f)
point_cloud_ds = threshold_point_cloud_seg_class(point_cloud, seg_classes=[4, 10, 12])
point_cloud_ds = threshold_point_cloud_depth(point_cloud_ds, max_depth=50.)
print('Downsampling')
# point_cloud = project_camera_points_by_class(seg_img, depth_img, fov, seg_classes=[4, 6, 7, 10, 12])
# point_cloud = project_camera_point_cloud(seg_img, depth_img, fov)
# for point in point_cloud:
#     plt.plot(point[0], point[2], '.', color=tuple([float(x) / 255. for x in point[3:]]) )

xmin_m = -10
xmax_m =  10
zmin_m =   0.
zmax_m =  50.

# Render as an image approach, Euclidean distance matrix approach not that fast.
#grid_res = 0.2
#point_cloud_top_view = downsample_and_project_point_cloud(point_cloud_ds, xmin_m, xmax_m, zmin_m, zmax_m, grid_res)           
#plt.imshow(point_cloud_top_view, origin='lower')

plt.scatter(point_cloud_ds[:, 0], point_cloud_ds[:,2], color=point_cloud_ds[:,4:]/255.)
plt.ylim([zmin_m, zmax_m])
plt.axis('equal')
plt.show()

Transform(Location(x=97.897575, y=191.629959, z=1.856224), Rotation(pitch=-0.174955, yaw=-1.334015, roll=-0.032135))
Projecting
Downsampling


In [ ]:
plt.figure()
# cam_points = project_camera_points(seg_img, depth_img, fov, seg_classes = [4, 10], final_transform=camera_transform)

cam_orig   = camera_transform.transform(carla.Location(0, 0, 0))
cam_x_axis = camera_transform.transform(carla.Location(75, 0, 0))
cam_y_axis = camera_transform.transform(carla.Location(0, 75, 0))

cam_x_axis_vg = local_to_world_frame(carla.Location(75, 0, 0), camera_transform)
cam_y_axis_vg = local_to_world_frame(carla.Location(0, 75, 0), camera_transform)
print('world: ', cam_x_axis_vg, cam_y_axis_vg)

x_recon = world_to_local_frame(cam_x_axis_vg, camera_transform)
y_recon = world_to_local_frame(cam_y_axis_vg, camera_transform)
print('local: ', x_recon, y_recon)

# for point in cam_points:
#     plot_color = tuple( [float(x) / 255. for x in COLORMAP[point[3]]] )
#     plt.plot(point[0], point[1], 'o', color=plot_color)

    
ego_x, ego_y = [0., 0.]

for (k,v) in data_dicts[img_num].items():
    
    if 'sensor' in k:
        [print(k, kk, vv) for (kk, vv) in v.items()]
        
    if 'vehicle' in k or 'walker' in k:
        plot_bbox_world_vertices(v['bounding_box']['world_vertices'])
        plot_orientation_arrow([v['location']['x'], v['location']['y']], \
                                v['rotation']['yaw'], \
                                1.0)
        if v['role_name'] == 'pedestrian':
            text = 'ped_%d' % v['id']
            text_col  = 'c'
        elif v['role_name'] == 'autopilot':
            text = 'veh_%d' % v['id']
            text_col  = 'b'
        elif v['role_name'] == 'hero':
            ego_x = v['location']['x']
            ego_y = v['location']['y']
            text = 'ego'
            text_col  = 'r'
        else:
            raise ValueError("Unexpected role: " % v['role_name'])
            
        plot_text([v['location']['x'], v['location']['y']], text, color=text_col)
    
    
plt.plot(camera_transform.location.x, camera_transform.location.y, 'bx')

plt.plot(cam_x_axis.x, cam_x_axis.y, 'gx')
plt.plot(cam_y_axis.x, cam_y_axis.y, 'yx')
plt.plot(cam_x_axis_vg[0], cam_x_axis_vg[1], 'g+')
plt.plot(cam_y_axis_vg[0], cam_y_axis_vg[1], 'y+')
    

In [ ]:
plt.figure()
im_overlay = np.copy(rgb_img)
# plt.imshow(rgb_img)

for (k,v) in data_dicts[img_num].items():
    if 'vehicle' in k or 'walker' in k:
        if v['id'] in [264]:
            pass
        else:
            continue
        img_height, img_width, _ = rgb_img.shape        
        us, vs = project_world_boxes_in_camera_frame(v['bounding_box'], camera_transform, img_width, img_height, fov)
        for ind, (u, v) in enumerate(zip(us, vs)):
            if u < 0 or v < 0 or u > img_width or v > img_height:
                pass
            else:
                u = int(u); v = int(v)
                cv2.circle(im_overlay, (u,v), 2,(255, 0, 0))
                
                
#         umin, vmin, umax, vmax = project_world_boxes_in_camera_frame(v['bounding_box'], camera_transform, img_width, img_height, fov)
        
#         if umax < 0 or vmax < 0:
#             pass # left/above image extent
#         elif umin > img_width or vmin > img_height:
#             pass # right/below image extent
#         else:
#             umin = max(0, int(umin))
#             vmin = max(0, int(vmin))
#             umax = min(img_width,  int(umax))
#             vmax = min(img_height, int(vmax))
            
#             if v['role_name'] == 'pedestrian':
#                 text = 'ped'
#                 bbox_color = COLORMAP[4]
#             elif v['role_name'] == 'autopilot':
#                 text = 'veh'
#                 bbox_color  = COLORMAP[10]        
#             elif v['role_name'] == 'hero':
#                 continue
#             else:
#                 raise ValueError("Unexpected role: s" % v['role_name'])

#             cv2.rectangle(im_overlay, (umin, vmin), (umax, vmax), bbox_color, 2)
#             cv2.putText(im_overlay, text, (umin+5, vmin+5), cv2.FONT_HERSHEY_SIMPLEX, 1, bbox_color, 2)

plt.imshow(im_overlay)
plt.show()

In [ ]:
plt.xlim([ego_x - 50., ego_x + 50.])
plt.ylim([ego_y - 50., ego_y + 50.])

plt.show()

In [ ]:
import carla
for (k,v) in data_dicts[img_num].items():
    if 'sensor' in k:
        loc = carla.Location(x = v['location']['x'],
                             y = v['location']['y'],
                             z = v['location']['z'])
        rot = carla.Rotation(pitch = v['rotation']['pitch'],
                             yaw   = v['rotation']['yaw'],
                             roll  = v['rotation']['roll'])
        trans = carla.Transform(location=loc, rotation=rot)
        
        # trans.transform(): local frame => global/map frame
        # trans.
        
        print(trans.location, trans.rotation)
        print(dir(trans))
        

# PROGRESS
* Shown how to convert segmentation/depth to a point cloud and project to top view.
* 

#TODO:
* 2D to 3D camera projection (need to get intrinsic matrix from json)
* camera <-> world frame transformation
* bbox plotting in world frame
* association of bbox with projected points
* 3D/2D bbox label for relevant objects (pedestrians, vehicles, stop/yield/speed signs, traffic lights)